In [30]:
from statsmodels.stats.contingency_tables import mcnemar
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz,process
from sklearn.metrics import confusion_matrix

In [43]:
sent_data_path = 'D:/PycharmProjects/MSThesisWorkMay2019/JournalCodes/data/GoldDataAllAspects_Annotate_Partial_17thSept.csv'
sent_data = pd.read_csv(sent_data_path)
summ_data_readpath = 'D:/Documents/MSThesisWork/AdobeBrandConsistency/genData1/GoldDataWithINFO_16thJune.csv'
summ_data = pd.read_csv(summ_data_readpath)
summ_data = summ_data.fillna("")

In [44]:
def gold_sent_match(gold_sent, aspect_list):
    for elem in aspect_list:
        if fuzz.partial_ratio(gold_sent, elem) > 90:
            return 1
    return 0

In [45]:
gold_tot_list = list()
authid_list = list()
prec_range_val = 3
lead_pre = list()
hybrid_pre = list()
incons_pre = list()
neg_pre = list()
ctr_pre = list()
incons_neg_pre = list()
rand_prec_list = []
topic_prec_list = []
fealexi_prec_list = []

In [46]:
for row_id in range(sent_data.shape[0]):
    
    gold_list1 = list(sent_data.iloc[row_id,3:(3+prec_range_val)])
    gold_list = [x for x in gold_list1 if x == x]
    if len(gold_list) < 3:
        print('Rowid: ', row_id, ' has less than 3 entries, having: ', len(gold_list))
    
    lead_list = list(sent_data.iloc[row_id, 6:(6+prec_range_val)])
    hybrid_list = list(sent_data.iloc[row_id, 9:(9+prec_range_val)])
    incons_list = list(sent_data.iloc[row_id, 12:(12+prec_range_val)])
    neg_list = list(sent_data.iloc[row_id, 15:(15+prec_range_val)])
    ctr_list = list(sent_data.iloc[row_id, 18:(18+prec_range_val)])
    incons_neg_list = list(sent_data.iloc[row_id, 21:(21+prec_range_val)])
    rand_list = list(summ_data.iloc[row_id, 9:(9+prec_range_val)])
    
    for elem in gold_list:
        authid_list.append(sent_data.iloc[row_id, 0])
        gold_tot_list.append(elem)
        lead_pre.append(gold_sent_match(elem, lead_list))
        hybrid_pre.append(gold_sent_match(elem, hybrid_list))
        incons_pre.append(gold_sent_match(elem, incons_list))
        neg_pre.append(gold_sent_match(elem, neg_list))
        ctr_pre.append(gold_sent_match(elem, ctr_list))
        incons_neg_pre.append(gold_sent_match(elem, incons_neg_list))
        rand_prec_list.append(gold_sent_match(elem, rand_list))
    
    
    
    
    
    

Rowid:  1  has less than 3 entries, having:  2
Rowid:  2  has less than 3 entries, having:  2
Rowid:  21  has less than 3 entries, having:  2
Rowid:  23  has less than 3 entries, having:  2
Rowid:  24  has less than 3 entries, having:  2
Rowid:  26  has less than 3 entries, having:  2
Rowid:  29  has less than 3 entries, having:  2
Rowid:  32  has less than 3 entries, having:  2
Rowid:  34  has less than 3 entries, having:  2


In [47]:
len(incons_pre)

99

In [48]:
cont_data = pd.DataFrame({'X.AUTHID': authid_list, 'GOLD': gold_tot_list, 'RAND': rand_prec_list, 'LEAD': lead_pre, 'HYBRID': hybrid_pre, 'INCONS': incons_pre, 'NEG': neg_pre, 'CTR': ctr_pre, 'INCONS_NEG': incons_neg_pre})
cont_data.to_csv('McnemarContTable_7thJuly.csv', index=False)

In [49]:
cont_data.head()

,X.AUTHID,GOLD,RAND,LEAD,HYBRID,INCONS,NEG,CTR,INCONS_NEG
0,b'http://www.mastec.com/press-release/1799/1/M...,"August 24, 2011;CORAL GABLES, Fla., Aug. 24, 2...",0,1,1,0,1,1,0
1,b'http://www.mastec.com/press-release/1799/1/M...,"As of ;, MasTec had appro ; in outstanding loa...",0,0,1,1,1,1,1
2,b'http://www.mastec.com/press-release/1799/1/M...,"MasTec\'s Chief E Officer, added, """"We are ver...",0,0,1,0,1,1,0
3,b'http://www.rpminc.com/news-releases/news-rel...,"RPM International Inc., a holding company, own...",0,0,1,1,1,1,1
4,b'http://www.rpminc.com/news-releases/news-rel...,"MEDINA, Ohio, April 3 /PRNewswire-FirstCall/ -...",0,1,0,0,0,0,0


In [37]:
conf_mat = 

[[47, 4], [7, 41]]

In [50]:
def mcnemar_test(conf_mat, first, second):
    # calculate mcnemar test
    result = mcnemar(conf_mat, exact=True)
    # summarize the finding
    print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
    # interpret the p-value
    alpha = 0.05
    if result.pvalue > alpha:
        print(first,' - ', second, ' :: Same proportions of errors (fail to reject H0)')
    else:
        print(first,' - ', second, ' :: Different proportions of errors (reject H0)')

In [38]:
mcnemar_test(confusion_matrix(list(cont_data['HYBRID']), list(cont_data['CTR'])).tolist(), 'Hybrid', 'Centrality')

statistic=4.000, p-value=0.549
Hybrid  -  Centrality  :: Same proportions of errors (fail to reject H0)


In [39]:
mcnemar_test(confusion_matrix(list(cont_data['HYBRID']), list(cont_data['NEG'])).tolist(), 'Hybrid', 'Polarity')

statistic=2.000, p-value=0.007
Hybrid  -  Polarity  :: Different proportions of errors (reject H0)


In [40]:
mcnemar_test(confusion_matrix(list(cont_data['HYBRID']), list(cont_data['INCONS_NEG'])).tolist(), 'Hybrid', 'InconsNeg')

statistic=9.000, p-value=0.087
Hybrid  -  InconsNeg  :: Same proportions of errors (fail to reject H0)


In [41]:
mcnemar_test(confusion_matrix(list(cont_data['HYBRID']), list(cont_data['INCONS'])).tolist(), 'Hybrid', 'Incons')

statistic=8.000, p-value=0.052
Hybrid  -  Incons  :: Same proportions of errors (fail to reject H0)


In [42]:
mcnemar_test(confusion_matrix(list(cont_data['HYBRID']), list(cont_data['LEAD'])).tolist(), 'Hybrid', 'Lead')

statistic=10.000, p-value=0.001
Hybrid  -  Lead  :: Different proportions of errors (reject H0)


In [51]:
mcnemar_test(confusion_matrix(list(cont_data['HYBRID']), list(cont_data['RAND'])).tolist(), 'Hybrid', 'Rand')

statistic=0.000, p-value=0.000
Hybrid  -  Rand  :: Different proportions of errors (reject H0)
